In [5]:
import os
import pdfplumber
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, Border, Side, Alignment

pdf_files = {
    "cardio_structured": "cardio_structured.pdf",
    "prot_sap_102": "prot_sap_102.pdf",
    "prot_sap_1": "prot_sap_1.pdf"
}

pages_to_extract = {
    "cardio_structured": 6,
    "prot_sap_102": 50,
    "prot_sap_1": 14
}

output_file = "tables.xlsx"
wb = Workbook()

def extract_table(pdf_path, page_number):
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[page_number - 1]  # Zero-based index
        table = page.extract_table()
        if table:
            return pd.DataFrame(table[1:], columns=table[0])  # Convert to DataFrame
        else:
            return None  

def get_border_style(pdf_name):
    if pdf_name == "cardio_structured":
        return Border(left=Side(style="thick"), right=Side(style="thick"),
                      top=Side(style="thick"), bottom=Side(style="thick"))
    elif pdf_name == "prot_sap_102":
        return Border(left=Side(style="thin"), right=Side(style="thin"),
                      top=Side(style="thin"), bottom=Side(style="thin"))
    else:
        return Border(left=Side(style="thin"), right=Side(style="thin"),
                      top=Side(style="thin"), bottom=Side(style="thin"))

def format_worksheet(ws, pdf_name):
    header_font = Font(name="Times New Roman", size=12, bold=False)  # Headers
    content_font = Font(name="Calibri", size=11, bold=False)         # Content
    bold_font = Font(name="Calibri", size=11, bold=True)             # Bold content
    alignment = Alignment(horizontal="center", vertical="center", wrap_text=True)

    border = get_border_style(pdf_name)

    for col in ws.columns:
        max_length = 0
        col_letter = col[0].column_letter
        for cell in col:
            if cell.value:
                max_length = max(max_length, len(str(cell.value)))

            if cell.column == 1:
                cell.alignment = Alignment(horizontal="left", vertical="center", wrap_text=True)
            else:
                cell.alignment = alignment

            cell.border = border

            if pdf_name == "prot_sap_102" and cell.row in [1, 2]:
                cell.font = bold_font
            else:
                
                if cell.row == 1 or cell.column == 1:
                    cell.font = header_font
                else:
                    cell.font = content_font

        
        col_width = max_length * 0.8
        col_width = min(max(col_width, 8), 20)
        ws.column_dimensions[col_letter].width = col_width

for sheet_name, pdf_path in pdf_files.items():
    page_num = pages_to_extract[sheet_name]
    print(f" Extracting from {sheet_name} (Page {page_num})...")

    df = extract_table(pdf_path, page_num)

    if df is not None:
        ws = wb.create_sheet(title=sheet_name.replace("_", " ").title())
        for r_idx, row in enumerate([df.columns] + df.values.tolist(), start=1):
            for c_idx, value in enumerate(row, start=1):
                ws.cell(row=r_idx, column=c_idx, value=value)
        format_worksheet(ws, sheet_name)
        print("Table extracted and formatted successfully!")
    else:
        print("No table detected. OCR might be needed.")

wb.remove(wb["Sheet"])

wb.save(output_file)
print(f"Extracted tables saved successfully as '{output_file}'")


 Extracting from cardio_structured (Page 6)...
Table extracted and formatted successfully!
 Extracting from prot_sap_102 (Page 50)...
Table extracted and formatted successfully!
 Extracting from prot_sap_1 (Page 14)...
Table extracted and formatted successfully!
Extracted tables saved successfully as 'tables.xlsx'
